# Load Necessary Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.common.exceptions import NoSuchElementException

### Assign Page Counter as pagRev 
### Assign Search URL to website we will be scraping
### Finally we will initalize dictionary to create structure of our dataset for the data we scrape

In [ ]:
pagRev = 0

search_url ='https://www.yelp.com/biz/golden-gate-park-san-francisco'

finalDataset = {
        'Name': [],
        'isElite': [],
        'bizRating': [],
        'dateReviewed': [],
        'BusName' : [],
        'BusCategory' : [],
        'BusReviewCounts' : [],
        'ParkingInfo': []
}    


# Create functions to easily scrape and export our data

 - __get_bus_meta:__  function that will scrape and return business name, business categories, '__Known for__' details, and Total business review counts  
 - __get_yelp_data( yelpObj ):__ function that takes in beautifulSoup object and grabs reviewer data. This includes reviewer user name, user elite status, reviewer rating between 1-5, and date when review was written  
 - __parse_page( next_url, pagin ):__ function that takes in a url to search and page counter. First, we set the maxium number of pages we would like to scrape from each business. This function will call our __get_yelp_data__ function to extract review data and then continue for each page until we reach last page or the maximum number of pages set  
 - __export_table_and_print( data ):__ function that takes in dictionary. This will transform our dictionary to a dataset and we will extract as a .csv file ***Make sure to rename csv file***

In [ ]:


def get_bus_meta(urlSame):

    driver = webdriver.Firefox()
    driver.get(urlSame)
    driver.implicitly_wait(100)
    tutorial_soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Create Values to return
    
    placeName = ""
    busReviewCount = ""
    busCats = ""
    testPark = {}
    parkingVal = []
    parkingKey = []
    
    
    
    ### Grab business Name
    grabBusName = tutorial_soup.findAll('div', attrs={'class': 'lemon--div__373c0__1mboc u-space-b1 border-color--default__373c0__2oFDT'})
    placeName = grabBusName[0].find('h1').text
    
    # grab categories
    grabCats = tutorial_soup.findAll('span',attrs={'class':'lemon--span__373c0__3997G display--inline__373c0__1DbOG u-space-r1 border-color--default__373c0__2oFDT'})
    busCats = grabCats[1].text

    
    # grab total bus reviews
    totalRevs = tutorial_soup.findAll('p',attrs={'class':'lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_ text-size--large__373c0__1568g'})
    print(totalRevs[0].text)
    busReviewCount = totalRevs[0].text

    
    tts = tutorial_soup.findAll('span', attrs={'class':'lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_'})
    print('key 1')
    print(tts[9].text)
    parkingKey.append(tts[9].text)
    print('key 2')    
    print(tts[10].text)    
    parkingKey.append(tts[10].text)    
    print('key 3')    
    print(tts[11].text) 
    parkingKey.append(tts[11].text)    
    print('key 4')    
    print(tts[12].text)        
    parkingKey.append(tts[12].text)
    

    ciclr =0
    tte = tutorial_soup.findAll('span', attrs={'class':'lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa'})
    for test123 in tte:
        ciclr = ciclr +1
        print(ciclr)
        print(test123.text)
    strSZ1 = len(tte[7].text)
    print(tte[7].text)
    parkingVal.append(tte[7].text[1:strSZ1])
    strSZ2 = len(tte[8].text)
    print(tte[8].text)
    parkingVal.append(tte[8].text[1:strSZ2])
    strSZ3 = len(tte[9].text) 
    print(tte[9].text)
    parkingVal.append(tte[9].text[1:strSZ3])
    strSZ4 = len(tte[10].text)
    print(tte[10].text)
    parkingVal.append(tte[10].text[1:strSZ4])

    
    for key in parkingKey: 
        for value in parkingVal: 
            testPark[key] = value 
            parkingVal.remove(value) 
            break

    # return values created
    return (placeName, busCats,testPark, busReviewCount)



def get_yelp_data(yelpObj):
    
    
    
    tew = yelpObj.findAll('div', attrs={'class': 'lemon--div__373c0__1mboc arrange__373c0__UHqhV gutter-6__373c0__zqA5A vertical-align-middle__373c0__2TQsQ border-color--default__373c0__2oFDT'})
    print('whole obj len')
    print(len(tew))
    for t in tew:
        testDate = t.find('span', attrs={'class': 'lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_'})
        testStar = t.findAll('span', attrs={'class':'lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT'})
        notUpdated = t.findAll('div', attrs= {'class':'lemon--div__373c0__1mboc display--inline-block__373c0__2de_K u-space-r-half border-color--default__373c0__2oFDT'})
        updat = t.findAll('span' , attrs = {'class':'lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa text-size--small__373c0__3SGMi'})

        
        if (len(updat)==1 and len(notUpdated)==1):
            finalDataset['dateReviewed'].append(testDate.text)
            finalDataset['bizRating'].append(testStar[0].find('div')['aria-label'][0])
        
        
        if(testDate and len(updat)==0 and len(notUpdated)==0):
            #print(testDate.text)
            print(notUpdated)
            finalDataset['dateReviewed'].append(testDate.text)


        if(testStar and len(updat)==0) and len(notUpdated)==0:
            print(testStar[0].find('div')['aria-label'])
            print(type(testStar[0].find('div')['aria-label']))
            finalDataset['bizRating'].append(testStar[0].find('div')['aria-label'][0])
    

             
    # grab user elite status and username        
   
    findusr = yelpObj.findAll('div', attrs={'class':'lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__2oFDT'})
    for usaInfo in findusr:
        #print(usaInfo.find('a').text)
        finalDataset['Name'].append(usaInfo.find('a').text)
    
    findElite = yelpObj.findAll('div', attrs={'class':'lemon--div__373c0__1mboc user-passport-stats__373c0__2LjLz border-color--default__373c0__2oFDT'})

    for eliInfo in findElite:
        if (eliInfo.find('a')):
            numOnly = eliInfo.find('a').text
            finalDataset['isElite'].append(numOnly.replace("Elite '",''))
            #print(numOnly.replace("Elite '",''))
        else: 
            #print(0)
            finalDataset['isElite'].append(0)
 

    print(finalDataset)




def parse_page(next_url, pagin):
    maxPg = 50
    
    # HTTP GET requests
    driver = webdriver.Firefox()
    driver.get(next_url)
    driver.implicitly_wait(100)
      
    pages_remaining = True
 
    while pages_remaining:
    # Extract reviews and check if theres a next page
        bs = BeautifulSoup(driver.page_source, 'lxml')
        
        get_yelp_data(bs)
        pagin = pagin + 1
        try:
            #check for max pages
            if(pagin > maxPg):
                print('Max Page reached!')
                pages_remaining = False
            #Checks if there are more pages with links
            next_link  = driver.find_element_by_link_text('Next')
            next_link.click()
            time.sleep(5)

        except (NoSuchElementException):
            print('Last Page Reached!')
            pages_remaining = False
            driver.quit()



def export_table_and_print(data):
    table = pd.DataFrame(data, columns=['Name','isElite','bizRating','dateReviewed',
                                        'BusName','BusCategory', 'BusReviewCounts', 'ParkingInfo'])
    table.index = table.index + 1
    # Print to see what we've created
    print(table)
    table.to_csv('yelpData.csv', sep=',', encoding='utf-8', index=False)




# Begin business data extraction

Since our __get_bus_meta__ function has data extracted in an array format we append it to our dataset for n amount of rows in our dataset

In [ ]:
parse_page(search_url, pagRev)
x = get_bus_meta(search_url)

szOfDtf = len(finalDataset['Name'])
print(len(x))
for i in range(szOfDtf):
    finalDataset['BusName'].append(x[0])
    finalDataset['BusCategory'].append(x[1])
    finalDataset['ParkingInfo'].append(x[2])
    finalDataset['BusReviewCounts'].append(x[3])

# Finally we extract our dataset to CSV file

In [ ]:

export_table_and_print(finalDataset)
